<a href="https://colab.research.google.com/github/rams1987/Essay_Scoring/blob/main/Essay_Scoring_Using_Pretrained_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install kaggle

!pip install kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"ramasamyseenivasagan","key":"f41bb0714d3baef12d51ba89162604ef"}'}

In [3]:
# prompt: create a kaggle folder

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c learning-agency-lab-automated-essay-scoring-2

learning-agency-lab-automated-essay-scoring-2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip learning-agency-lab-automated-essay-scoring-2.zip

Archive:  learning-agency-lab-automated-essay-scoring-2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import os
import numpy as np
import pandas as pd
import spacy

In [7]:
essay_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

essay_df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [8]:
essays = essay_df['full_text'].tolist()
labels = essay_df['score'].tolist()

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=1)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [10]:
training_args = TrainingArguments(
      output_dir="training",
      logging_dir='logs',
      logging_strategy='epoch',
      logging_steps=100,
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=64,
      weight_decay=0.01,
  )

trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=essays,
      eval_dataset=labels,
      compute_metrics=None,
      tokenizer=tokenizer,
  )

trainer.train()

AttributeError: 'list' object has no attribute 'keys'

In [9]:
# Preprocess the essays using the tokenizer from step 1
encoded_inputs = tokenizer(essays, truncation=True, padding="max_length")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
print(len(essays))
print(len(encoded_inputs))
print(len(labels))

17307
3
17307


In [10]:
# prompt: Split essay_df into train and test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_inputs, labels, test_size=0.2, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [3, 17307]

In [24]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [26]:
valid_indices = X_train.index[X_train.index.isin(X_test.index)]
X_train = X_train[valid_indices]

In [10]:
pip install accelerate -U

In [27]:
# prompt: build a model using Debertav3 for essay scoring

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=1)

with tf.device('/GPU:0'):  # Use the first GPU device by default

  training_args = TrainingArguments(
      output_dir="training",
      logging_dir='logs',
      logging_strategy='epoch',
      logging_steps=100,
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=64,
      weight_decay=0.01,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=X_train,
      eval_dataset=X_test,
      compute_metrics=None,
      tokenizer=tokenizer,
  )

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# prompt: Test the model against the test dataset

y_pred = trainer.predict(test_df['full_text']).predictions.argmax(-1)
